In [1]:
pip install faiss-cpu

   ---------------------------------------- 0.0/18.8 MB ? eta -:--:--
   -------- ------------------------------- 3.9/18.8 MB 23.5 MB/s eta 0:00:01
   ----------------------- ---------------- 11.0/18.8 MB 29.9 MB/s eta 0:00:01
   ---------------------------------------  18.6/18.8 MB 34.6 MB/s eta 0:00:01
   ---------------------------------------- 18.8/18.8 MB 32.9 MB/s  0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import faiss

In [3]:
faiss.__version__

'1.13.1'

In [4]:
bio_text = """
Making an Impact
Helping Millions of Students Succeed
Sudhanshu's commitment to affordable education wasn't just a business strategy—it was his life's mission. Over the years, iNeuron has helped over 1.5 million students from 34+ countries, providing them with the skills they need to succeed in today's competitive job market. Many of these students, like Sudhanshu himself, came from disadvantaged backgrounds. They saw iNeuron as a lifeline—an opportunity to rise above their circumstances.

In 2022, iNeuron was acquired by PhysicsWallah in a deal worth ₹250 crore. While this acquisition was a significant milestone, Sudhanshu remained focused on his mission. Even after the acquisition, iNeuron continued to offer some of the most affordable and accessible tech courses in the world.

The Entrepreneur and Teacher: Sudhanshu's Dual Legacy
Sudhanshu's journey isn't just one of entrepreneurial success; it's also a story of dedication to teaching. Throughout his career, he has remained a passionate educator, constantly looking for ways to empower others through knowledge. Whether teaching courses in Big Data, Data Science, or programming, Sudhanshu has always sought to make complex subjects accessible to learners at all levels.

His commitment to affordable education has earned him the respect and admiration of countless students. Many credit Sudhanshu with changing their lives, helping them secure jobs, improve their skills, and break free from the limitations of their backgrounds.

Th
e Journey

Early Life and the Power of Education

Rising Through the Ranks of the Tech World

The Birth of iNeuron: Democratizing Education

The Foundation of Euron: Expanding the Mission

Early Life and the Power of Education
Sudhanshu Kumar's life is a story of triumph over adversity, driven by the belief in the transformative power of education. Born in Jamshedpur, Jharkhand, India, to a family of very modest means, Sudhanshu's early years were marked by financial hardship. His surroundings offered little opportunity, and resources were limited, yet he understood from a young age that education could be his ticket out of poverty.

While many would have been daunted by the lack of support and opportunity, Sudhanshu was relentless in his pursuit of knowledge. He knew that education had the power to change lives, and he was determined to leverage it to create a better future for himself and his family. Despite the numerous challenges along the way, Sudhanshu excelled academically, eventually earning a degree in Computer Science and Engineering (CSE).
"""

In [5]:
import requests
import os
import json
import numpy as np

In [7]:
def chunk_text(text, chunk_size=10, overlap=2):
    words = text.split()
    chunks = []
    start = 0
    while start < len(words):
        end = min(start + chunk_size, len(words))
        chunk = ' '.join(words[start:end])
        chunks.append(chunk)
        start += chunk_size - overlap
    return chunks

In [9]:
chunks = chunk_text(bio_text)

In [10]:
chunks

["Making an Impact Helping Millions of Students Succeed Sudhanshu's commitment",
 "Sudhanshu's commitment to affordable education wasn't just a business strategy—it",
 "business strategy—it was his life's mission. Over the years, iNeuron",
 'years, iNeuron has helped over 1.5 million students from 34+',
 'from 34+ countries, providing them with the skills they need',
 "they need to succeed in today's competitive job market. Many",
 'market. Many of these students, like Sudhanshu himself, came from',
 'came from disadvantaged backgrounds. They saw iNeuron as a lifeline—an',
 'a lifeline—an opportunity to rise above their circumstances. In 2022,',
 'In 2022, iNeuron was acquired by PhysicsWallah in a deal',
 'a deal worth ₹250 crore. While this acquisition was a',
 'was a significant milestone, Sudhanshu remained focused on his mission.',
 'his mission. Even after the acquisition, iNeuron continued to offer',
 'to offer some of the most affordable and accessible tech',
 'accessible tech 

In [11]:
API_URL = "https://api.euron.one/api/v1/euri/embeddings"
API_KEY = "euri-f8fd85b48e1345aff339a73271b309ac30058e036059bf5cdf316ad4cac40c4a"  # Replace with your actual API key
MODEL_NAME = "text-embedding-3-small"

In [12]:
header = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {API_KEY}"
}

In [13]:
all_embeddings = []
for i ,chunk in enumerate(chunks):
    payload = {
        "model": MODEL_NAME,
        "input": chunk
    }
    response = requests.post(API_URL, headers=header, data=json.dumps(payload))
    data = response.json()
    embeddings = data['data'][0]['embedding']
    all_embeddings.append(embeddings)
    
    

In [14]:
len(all_embeddings)

49

In [15]:
len(chunks)

49

In [16]:
type(all_embeddings)

list

In [17]:
embeddings_array = np.array(all_embeddings,dtype='float32')

In [18]:
embeddings_array

array([[ 0.01370199, -0.03450083,  0.06028342, ..., -0.00386407,
        -0.0038541 , -0.00483091],
       [-0.00270775,  0.02011846,  0.0172799 , ..., -0.00195887,
        -0.00337815, -0.00348607],
       [-0.00975724,  0.01606824, -0.00957996, ..., -0.01460749,
         0.00899141, -0.02998082],
       ...,
       [ 0.06212645,  0.01672526,  0.05411197, ..., -0.03163163,
        -0.05351514, -0.0124125 ],
       [-0.00824434, -0.0119207 ,  0.03207352, ..., -0.01180467,
         0.02070244, -0.00081833],
       [-0.00381053,  0.00636478,  0.03010318, ..., -0.06082558,
        -0.00293381,  0.01994574]], dtype=float32)

In [19]:
base_index = faiss.IndexFlatL2(1536)

In [21]:
base_index.add(embeddings_array)

In [22]:
faiss.write_index(base_index, "faiss_index.faiss")

In [23]:
query_test = "tell me about kumar early life"

In [24]:
def embedding_text(text):
    payload = {
        "model": MODEL_NAME,
        "input": text
    }
    response = requests.post(API_URL, headers=header, data=json.dumps(payload))
    data = response.json()
    embeddings = data['data'][0]['embedding']
    emb = np.array(embeddings,dtype="float32").reshape(1,-1)

    return emb

In [25]:
query_test_emb  = embedding_text(query_test)

In [27]:
query_test_emb

array([[-0.00490735, -0.01988584, -0.04520792, ...,  0.04454121,
         0.02555286,  0.00792357]], dtype=float32)

In [28]:
base_index.search(query_test_emb,3)

(array([[1.1385286, 1.1385286, 1.1486495]], dtype=float32),
 array([[32, 81, 27]], dtype=int64))

In [29]:
chunks[4]

'from 34+ countries, providing them with the skills they need'

In [30]:
chunks[8]

'a lifeline—an opportunity to rise above their circumstances. In 2022,'

In [31]:
chunks[6]

'market. Many of these students, like Sudhanshu himself, came from'